## Data preparation and cleaning



In [20]:
import pandas as pd
import sqlite3

# Load the dataset
dataset_path = 'Customer_support_data.csv'  # Make sure the file path matches where you've uploaded your file in Colab
data = pd.read_csv(dataset_path)

# Optionally rename columns for consistency or clarity (example provided)
data.rename(columns={'Customer Remarks': 'Customer_Remarks', 'CSAT Score': 'CSAT_Score'}, inplace=True)

# Print the first few rows of the dataset to confirm it's loaded correctly
print("First few rows of the dataset:")
print(data.head())

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('customer_support.db')

# Create a table named 'customer_support' and import the DataFrame into the database
# If the table already exists, 'replace' it
data.to_sql('customer_support', conn, if_exists='replace', index=False)

# Using a cursor to query the database and print each row in the 'customer_support' table
cur = conn.cursor()
print("\nSample rows from the 'customer_support' table:")
for row in cur.execute('SELECT * FROM customer_support LIMIT 5'):  # Adjust the LIMIT for more or fewer rows
    print(row)

# Close the connection to the database
conn.close()

# Confirm that the database operations are complete
print("\nDatabase operations completed successfully.")


First few rows of the dataset:
                              Unique id channel_name         category  \
0  7e9ae164-6a8b-4521-a2d4-58f7c9fff13f      Outcall  Product Queries   
1  b07ec1b0-f376-43b6-86df-ec03da3b2e16      Outcall  Product Queries   
2  200814dd-27c7-4149-ba2b-bd3af3092880      Inbound    Order Related   
3  eb0d3e53-c1ca-42d3-8486-e42c8d622135      Inbound          Returns   
4  ba903143-1e54-406c-b969-46c52f92e5df      Inbound     Cancellation   

                   Sub-category Customer_Remarks  \
0                Life Insurance              NaN   
1  Product Specific Information              NaN   
2             Installation/demo              NaN   
3        Reverse Pickup Enquiry              NaN   
4                    Not Needed              NaN   

                               Order_id order_date_time Issue_reported at  \
0  c27c9bb4-fa36-4140-9f1f-21009254ffdb             NaN  01/08/2023 11:13   
1  d406b0c7-ce17-4654-b9de-f08d421254bd             NaN  01/08/

In [21]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('customer_support.db')

# Load data from the customer_support table into a DataFrame
customer_support_df = pd.read_sql_query("SELECT * FROM customer_support", conn)

# Print DataFrame information to get a summary including the number of non-null entries per column
print("DataFrame Information:")
print(customer_support_df.info())

# Generate and print descriptive statistics for numeric columns
print("\nDescriptive Statistics:")
print(customer_support_df.describe())

# Print the first few rows of the DataFrame to get an idea of the data structure
print("\nFirst 5 Rows of the DataFrame:")
print(customer_support_df.head())

# Count and print the number of missing values in each column
print("\nCount of Null Values in Each Column:")
print(customer_support_df.isnull().sum())

num_rows_no_nulls = customer_support_df.dropna().shape[0]
print(f"Number of rows with no null values: {num_rows_no_nulls}")

total_rows = customer_support_df.shape[0]
print(f"Total number of rows: {total_rows}")
# Close the database connection
conn.close()


DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unique id                85907 non-null  object 
 1   channel_name             85907 non-null  object 
 2   category                 85907 non-null  object 
 3   Sub-category             85907 non-null  object 
 4   Customer_Remarks         28756 non-null  object 
 5   Order_id                 67675 non-null  object 
 6   order_date_time          17214 non-null  object 
 7   Issue_reported at        85907 non-null  object 
 8   issue_responded          85907 non-null  object 
 9   Survey_response_Date     85907 non-null  object 
 10  Customer_City            17079 non-null  object 
 11  Product_category         17196 non-null  object 
 12  Item_price               17206 non-null  float64
 13  connected_handling_time  242 non-null    float64
 14 

In [22]:
import pandas as pd

# Assuming your dataframe is named 'customer_support_df' and has been loaded correctly

# Convert the columns to datetime
customer_support_df['order_date_time'] = pd.to_datetime(customer_support_df['order_date_time'])
customer_support_df['Issue_reported at'] = pd.to_datetime(customer_support_df['Issue_reported at'])
customer_support_df['issue_responded'] = pd.to_datetime(customer_support_df['issue_responded'])
customer_support_df['Survey_response_Date'] = pd.to_datetime(customer_support_df['Survey_response_Date'])

# Calculate response time (in whatever time unit is appropriate, e.g., minutes)
customer_support_df['response_time_minutes'] = (customer_support_df['issue_responded'] - customer_support_df['Issue_reported at']).dt.total_seconds() / 60

# Extract hour of day for order and issue reported time
customer_support_df['order_hour_of_day'] = customer_support_df['order_date_time'].dt.hour
customer_support_df['issue_reported_hour_of_day'] = customer_support_df['Issue_reported at'].dt.hour

# Extract day of the week (0=Monday, 6=Sunday)
customer_support_df['order_day_of_week'] = customer_support_df['order_date_time'].dt.dayofweek
customer_support_df['issue_reported_day_of_week'] = customer_support_df['Issue_reported at'].dt.dayofweek

# Calculate elapsed time from order to survey response (in days)
customer_support_df['elapsed_days_to_survey'] = (customer_support_df['Survey_response_Date'] - customer_support_df['order_date_time']).dt.total_seconds() / (60 * 60 * 24)

# Now, your dataframe will have the new columns with the calculated data
# Print DataFrame information to get a summary including the number of non-null entries per column
print("DataFrame Information:")
print(customer_support_df.info())

# Generate and print descriptive statistics for numeric columns
print("\nDescriptive Statistics:")
print(customer_support_df.describe())

# Print the first few rows of the DataFrame to get an idea of the data structure
print("\nFirst 5 Rows of the DataFrame:")
print(customer_support_df.head())

# Count and print the number of missing values in each column
print("\nCount of Null Values in Each Column:")
print(customer_support_df.isnull().sum())

num_rows_no_nulls = customer_support_df.dropna().shape[0]
print(f"Number of rows with no null values: {num_rows_no_nulls}")

total_rows = customer_support_df.shape[0]
print(f"Total number of rows: {total_rows}")
# Close the database connection
conn.close()


DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unique id                   85907 non-null  object        
 1   channel_name                85907 non-null  object        
 2   category                    85907 non-null  object        
 3   Sub-category                85907 non-null  object        
 4   Customer_Remarks            28756 non-null  object        
 5   Order_id                    67675 non-null  object        
 6   order_date_time             17214 non-null  datetime64[ns]
 7   Issue_reported at           85907 non-null  datetime64[ns]
 8   issue_responded             85907 non-null  datetime64[ns]
 9   Survey_response_Date        85907 non-null  datetime64[ns]
 10  Customer_City               17079 non-null  object        
 11  Product_category            171

In [23]:
# Drop the columns with high null values
customer_support_df.drop(columns=[
    'order_date_time',
    'order_hour_of_day',
    'order_day_of_week',
    'elapsed_days_to_survey',
    'Order_id',
    'Customer_City',
    'Item_price',
    'connected_handling_time',
    'Product_category'
], inplace=True)

# Print the remaining columns and the count of null values in each
print("Remaining columns and the number of null values in each:")
print(customer_support_df.isnull().sum())

num_rows_no_nulls = customer_support_df.dropna().shape[0]
print(f"Number of rows with no null values: {num_rows_no_nulls}")

Remaining columns and the number of null values in each:
Unique id                         0
channel_name                      0
category                          0
Sub-category                      0
Customer_Remarks              57151
Issue_reported at                 0
issue_responded                   0
Survey_response_Date              0
Agent_name                        0
Supervisor                        0
Manager                           0
Tenure Bucket                     0
Agent Shift                       0
CSAT_Score                        0
response_time_minutes             0
issue_reported_hour_of_day        0
issue_reported_day_of_week        0
dtype: int64


In [27]:
import pandas as pd
import sqlite3

# Assuming your original DataFrame is named 'customer_support_df'

# Filter the DataFrame to include only rows with no missing values
complete_cases_df = customer_support_df.dropna()

# Connect to a new SQLite database (this will create the database if it doesn't exist)
conn = sqlite3.connect('cleaned_customer_support.db')

# Save the filtered DataFrame as a new table in the SQLite database
complete_cases_df.to_sql('cleaned_customer_support', conn, if_exists='replace', index=False)

# Close the connection
conn.close()



Cleaned database with complete cases has been created.


In [28]:
import pandas as pd
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('cleaned_customer_support.db')

# Load the table into a pandas DataFrame
cleaned_df = pd.read_sql_query("SELECT * FROM cleaned_customer_support", conn)

# Print basic statistics of the cleaned database
print("Basic statistics of the cleaned database:")
print(cleaned_df.describe())

# Print information about the DataFrame
print("\nInformation about the DataFrame:")
cleaned_df.info()

# Close the database connection
conn.close()


Basic statistics of the cleaned database:
         CSAT_Score  response_time_minutes  issue_reported_hour_of_day  \
count  28756.000000           28756.000000                28756.000000   
mean       4.073619             324.363159                   14.457087   
std        1.559810           20793.037491                    5.066877   
min        1.000000         -295198.000000                    0.000000   
25%        4.000000               2.000000                   11.000000   
50%        5.000000               5.000000                   15.000000   
75%        5.000000              27.000000                   19.000000   
max        5.000000          177060.000000                   23.000000   

       issue_reported_day_of_week  
count                28756.000000  
mean                     2.984386  
std                      2.006198  
min                      0.000000  
25%                      1.000000  
50%                      3.000000  
75%                      5.000000  
max